In [ ]:
import gdown
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pprint
import random
import pandas as pd
import pulp
import itertools
import requests
import random
from geopy.distance import geodesic
import time
import pandas as pd
import os

Downloading...
From (original): https://drive.google.com/uc?id=1zq17RENYLU1qrE2gNPsxRAPoysWMZAqf
From (redirected): https://drive.google.com/uc?id=1zq17RENYLU1qrE2gNPsxRAPoysWMZAqf&confirm=t&uuid=721ad025-5c55-4361-ba41-60dfe4802e17
To: /content/mpsis_setup.py
100%|██████████| 12.1k/12.1k [00:00<00:00, 12.0MB/s]


Mounted at /content/drive


In [ ]:
def generate_random_point(sw_lat, sw_lng, ne_lat, ne_lng):
    lat = random.uniform(sw_lat, ne_lat)
    lng = random.uniform(sw_lng, ne_lng)
    return lat, lng

def snap_to_road(lat, lng, api_key):
    url = f'https://roads.googleapis.com/v1/nearestRoads?points={lat},{lng}&key={api_key}'
    response = requests.get(url)
    data = response.json()
    if 'snappedPoints' in data and len(data['snappedPoints']) > 0:
        snapped_point = data['snappedPoints'][0]['location']
        return snapped_point['latitude'], snapped_point['longitude']
    return None

def get_road_distance_time(origin, destination, api_key):
    origin_str = f'{origin[0]}, {origin[1]}'
    destination_str = f'{destination[0]}, {destination[1]}'
    departure_time = int(time.time()) + 24 * 3600
    url = (
        f'https://maps.googleapis.com/maps/api/distancematrix/json?units=metric&origins={origin_str}'
        f'&destinations={destination_str}&mode=driving&departure_time={departure_time}&key={api_key}'
    )
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        element = data['rows'][0]['elements'][0]
        if element['status'] == 'OK':
            distance_text = element['distance']['text']
            distance = float(distance_text.replace(' km', '').replace(',', ''))

            duration_text = element['duration']['text']
            duration_parts = duration_text.split()
            duration = int(duration_parts[0])
            if "hour" in duration_text:
                duration = int(duration_parts[0]) * 60 + int(duration_parts[2])

            return distance, duration
        else:
            return None, None
    else:
        return None, None

def find_center(sw_lat, sw_lng, ne_lat, ne_lng):
    center_lat = (sw_lat + ne_lat) / 2
    center_lng = (sw_lng + ne_lng) / 2
    return center_lat, center_lng

def generate_list(n, target_sum):
    if n <= 1:
        raise ValueError("The length of the list must be greater than 1.")
    if target_sum < 0:
        raise ValueError("The target sum must be non-negative.")
    result = [0]
    random_values = np.random.rand(n - 1)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    result.extend(rounded_values)

    return result

def crop_matrix(data, N):
    return [row[:N] for row in data[:N]]

import numpy as np

def generate_demand_list(n, m, target_sum):
    if n <= 1:
        raise ValueError("The length of the list must be greater than 1.")
    if m >= n:
        raise ValueError("M must be less than N.")
    if target_sum < 0:
        raise ValueError("The target sum must be non-negative.")

    random_values = np.random.rand(n - m - 1)  # n - (m + 1) non-zero elements
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    zeros_list = [0] * m

    result = list(rounded_values) + zeros_list

    random.shuffle(result)

    result = [-1] + result

    return result
def replace_zeros_with_sum(input_list, target_sum):
    zero_positions = [i for i, value in enumerate(input_list) if value == 0]
    num_zeros = len(zero_positions)

    result = input_list.copy()

    if num_zeros == 0:
        raise ValueError("There are no zeros to replace in the list.")

    random_values = np.random.rand(num_zeros)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    for i, pos in enumerate(zero_positions):
        result[pos] = rounded_values[i]

    return result

def create_path_as_list(pairs_list):
    wanted_length = len(pairs_list)
    path = ''
    current_pair = next(pair for pair in pairs_list if pair[0] == 0)
    path += str(current_pair[0])

    next_start = current_pair[1]

    pairs_list.remove(current_pair)

    while len(path) < wanted_length:
        path += str(next_start)
        current_pair = next(pair for pair in pairs_list if pair[0] == next_start)
        pairs_list.remove(current_pair)
        next_start = current_pair[1]

    return path

def find_paths_vrb(edges, k):
    def find_path_vrb(current_path):
        if len(paths) >= k:
            return

        last_node = current_path[-1]

        if last_node == 0 and len(current_path) > 1:
            paths.append(current_path)
            return

        for edge in edges:
            if edge[0] == last_node:
                find_path_vrb(current_path + [edge[1]])

    paths = []
    find_path_vrb([0])

    return paths

def repetitions(list1, list2):
  common_elements = set(list1) & set(list2)
  return (2*len(common_elements))/len(list1+list2)

def equal_paths(list1, list2):
  are_equal = sorted(list1) == sorted(list2)
  if are_equal:
    return 1
  else:
    return 0

In [ ]:
def vrpsdp(number_of_clients, number_of_vehicles, cost_buffor, demand_of_client_delivery, demand_of_client_pickup, file_idx, problem_idx):
  try:
      cost_shiffted = crop_matrix(cost_buffor, number_of_clients + 1)

      vrpsdp = pulp.LpProblem("VRPSDP", pulp.LpMinimize)
      m = number_of_vehicles
      x = pulp.LpVariable.dicts("x", [(i, j, k) for i in range(number_of_clients+1) for j in range(number_of_clients+1) for k in range(m)], cat='Binary')
      y = pulp.LpVariable.dicts("y", [(i, j) for i in range(number_of_clients+1) for j in range(number_of_clients+1)], lowBound=0, cat='Continuous')
      z = pulp.LpVariable.dicts("z", [(i, j) for i in range(number_of_clients+1) for j in range(number_of_clients+1)], lowBound=0, cat='Continuous')

      vrpsdp += pulp.lpSum(cost_shiffted[i][j] * x[i, j, k] for k in range(m) for i in range(number_of_clients+1) for j in range(number_of_clients+1) if i != j)
      p = demand_of_client_pickup
      d = demand_of_client_delivery
      Q = capacity_of_vehicle

      for j in range(1, number_of_clients+1):
          vrpsdp += pulp.lpSum(x[i, j, k] for k in range(m) for i in range(number_of_clients+1) if i != j) == 1

      for k in range(m):
        for j in range(number_of_clients+1):
          vrpsdp += pulp.lpSum(x[i,j,k] for i in range(number_of_clients+1)) - pulp.lpSum(x[j,i,k] for i in range(number_of_clients+1)) == 0

      for k in range(m):
        vrpsdp += pulp.lpSum(x[0,j,k] for j in range(1, number_of_clients+1)) == 1

      for j in range(1, number_of_clients+1):
        vrpsdp += pulp.lpSum(y[j,i] for i in range(number_of_clients+1)) - pulp.lpSum(y[i,j] for i in range(number_of_clients+1)) == p[j]

      for j in range(1, number_of_clients+1):
        vrpsdp += pulp.lpSum(z[i,j] for i in range(number_of_clients+1)) - pulp.lpSum(z[j,i] for i in range(number_of_clients+1)) == d[j]

      for i in range(number_of_clients+1):
        for j in range(number_of_clients+1):
          if i != j:
            vrpsdp += y[i,j] + z[i,j] <= Q * pulp.lpSum(x[i,j,k] for k in range(m))

      vrpsdp += pulp.lpSum(x[0,j,k] for j in range(1, number_of_clients+1) for k in range(m)) == m
      for i in range(1,number_of_clients+1):
        vrpsdp += pulp.lpSum(x[i,j,k] for j in range(number_of_clients+1) for k in range(m) if i != j) == 1

      vrpsdp.solve()

      all_uesd_edges = []
      paths = [[] for _ in range(number_of_vehicles)]

      for k in range(number_of_vehicles):
          for i in range(number_of_clients + 1):
              for j in range(number_of_clients + 1):
                  if i != j and pulp.value(x[i,j,k]) == 1:
                      all_uesd_edges.append((i, j))
                      paths[k].append((i, j))
      result_paths = find_paths_vrb(all_uesd_edges, number_of_vehicles)
      print(result_paths)

      return pulp.value(vrpsdp.objective), result_paths, all_uesd_edges

  except Exception as e:
      print(f"error for delivery stops {number_of_clients} and vehicles {number_of_vehicles}")

      values_to_reload.append((number_of_clients, number_of_vehicles, file_idx, problem_idx))

      return None, None, None

In [ ]:
def vrpsdp_error(number_of_clients, number_of_vehicles, cost_buffor, demand_of_client_delivery, demand_of_client_pickup, file_idx):
  try:
      cost_shiffted = crop_matrix(cost_buffor, number_of_clients + 1)
      vrpsdp = pulp.LpProblem("VRPSDP", pulp.LpMinimize)
      m = number_of_vehicles
      x = pulp.LpVariable.dicts("x", [(i, j, k) for i in range(number_of_clients+1) for j in range(number_of_clients+1) for k in range(m)], cat='Binary')
      y = pulp.LpVariable.dicts("y", [(i, j) for i in range(number_of_clients+1) for j in range(number_of_clients+1)], lowBound=0, cat='Continuous')
      z = pulp.LpVariable.dicts("z", [(i, j) for i in range(number_of_clients+1) for j in range(number_of_clients+1)], lowBound=0, cat='Continuous')

      vrpsdp += pulp.lpSum(cost_shiffted[i][j] * x[i, j, k] for k in range(m) for i in range(number_of_clients+1) for j in range(number_of_clients+1) if i != j)
      p = demand_of_client_pickup
      d = demand_of_client_delivery
      Q = capacity_of_vehicle

      for j in range(1, number_of_clients+1):
          vrpsdp += pulp.lpSum(x[i, j, k] for k in range(m) for i in range(number_of_clients+1) if i != j) == 1

      for k in range(m):
        for j in range(number_of_clients+1):
          vrpsdp += pulp.lpSum(x[i,j,k] for i in range(number_of_clients+1)) - pulp.lpSum(x[j,i,k] for i in range(number_of_clients+1)) == 0

      for k in range(m):
        vrpsdp += pulp.lpSum(x[0,j,k] for j in range(1, number_of_clients+1)) == 1

      for j in range(1, number_of_clients+1):
        vrpsdp += pulp.lpSum(y[j,i] for i in range(number_of_clients+1)) - pulp.lpSum(y[i,j] for i in range(number_of_clients+1)) == p[j]

      for j in range(1, number_of_clients+1):
        vrpsdp += pulp.lpSum(z[i,j] for i in range(number_of_clients+1)) - pulp.lpSum(z[j,i] for i in range(number_of_clients+1)) == d[j]

      for i in range(number_of_clients+1):
        for j in range(number_of_clients+1):
          if i != j:
            vrpsdp += y[i,j] + z[i,j] <= Q * pulp.lpSum(x[i,j,k] for k in range(m))

      vrpsdp += pulp.lpSum(x[0,j,k] for j in range(1, number_of_clients+1) for k in range(m)) == m
      for i in range(1,number_of_clients+1):
        vrpsdp += pulp.lpSum(x[i,j,k] for j in range(number_of_clients+1) for k in range(m) if i != j) == 1

      vrpsdp.solve()

      all_uesd_edges = []
      paths = [[] for _ in range(number_of_vehicles)]

      for k in range(number_of_vehicles):
          for i in range(number_of_clients + 1):
              for j in range(number_of_clients + 1):
                  if i != j and pulp.value(x[i,j,k]) == 1:
                      all_uesd_edges.append((i, j))
                      paths[k].append((i, j))
      result_paths = find_paths_vrb(all_uesd_edges, number_of_vehicles)

      return pulp.value(vrpsdp.objective), result_paths, all_uesd_edges

  except Exception as e:
      print(f"ONE MORE TIME ERROR")
      return None, None, None

In [ ]:
def create_cost_matrixes(df):
    real_distance_columns = [col for col in df.columns if col.startswith('Road Distance to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_raod = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Road Distance to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_raod[i][j] = float(distance)


    real_distance_columns = [col for col in df.columns if col.startswith('Time to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_time = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Time to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_time[i][j] = float(distance)


    real_distance_columns = [col for col in df.columns if col.startswith('Real Distance to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_real = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Real Distance to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_real[i][j] = float(distance)

    return cost_raod, cost_time, cost_real

### CRACOW

In [ ]:
# files = ['30_clients_cracow_1', '30_clients_cracow_2']#, '30_clients_cracow_3.csv','30_clients_cracow_4.csv', '30_clients_cracow_5.csv', '30_clients_cracow_6.csv', '30_clients_cracow_7.csv', '30_clients_cracow_8.csv', '30_clients_cracow_9.csv', '30_clients_cracow_10.csv', '30_clients_cracow_11.csv', '30_clients_cracow_12.csv', '30_clients_cracow_13.csv', '30_clients_cracow_14.csv', '30_clients_cracow_15.csv', '30_clients_cracow_16.csv', '30_clients_cracow_17.csv', '30_clients_cracow_18.csv', '30_clients_cracow_19.csv', '30_clients_cracow_20.csv']

['30_clients_cracow_1', '30_clients_cracow_2', '30_clients_cracow_3.csv', '30_clients_cracow_4.csv', '30_clients_cracow_5.csv', '30_clients_cracow_6.csv', '30_clients_cracow_7.csv', '30_clients_cracow_8.csv', '30_clients_cracow_9.csv', '30_clients_cracow_10.csv', '30_clients_cracow_11.csv', '30_clients_cracow_12.csv', '30_clients_cracow_13.csv', '30_clients_cracow_14.csv', '30_clients_cracow_15.csv', '30_clients_cracow_16.csv', '30_clients_cracow_17.csv', '30_clients_cracow_18.csv', '30_clients_cracow_19.csv', '30_clients_cracow_20.csv']


In [ ]:
capacity_of_vehicle = 100

objectiv_function_road = [[[] for _ in range(3)] for _ in range(3)]
objectiv_function_time = [[[] for _ in range(3)] for _ in range(3)]
objectiv_function_real = [[[] for _ in range(3)] for _ in range(3)]
all_paths_road = [[[] for _ in range(3)] for _ in range(3)]
all_paths_time = [[[] for _ in range(3)] for _ in range(3)]
all_paths_real = [[[] for _ in range(3)] for _ in range(3)]
all_uesd_edges_road = [[[] for _ in range(3)] for _ in range(3)]
all_uesd_edges_time = [[[] for _ in range(3)] for _ in range(3)]
all_uesd_edges_real = [[[] for _ in range(3)] for _ in range(3)]

values_to_reload = []
vechicles = [1,2,3]
clients = [10,10,10]

for requested_file in files:
  print(f"############# calculated_file {requested_file}")
  file_idx = files.index(requested_file)
  df = pd.read_csv(os.path.join(output_dir, requested_file))
  df.fillna(10000000, inplace=True)
  cost_R, cost_T, cost_G = create_cost_matrixes(df)

  v_iterator = 0

  for number_of_vehicles in vechicles:
    c_iterator = 0

    for number_of_clients in clients:

      print(f"calculate for {number_of_clients} clients and {number_of_vehicles} vechicles")

      demand_of_client_delivery = generate_list(number_of_clients + 1, 0.6 * capacity_of_vehicle * number_of_vehicles)
      demand_of_client_pickup = generate_list(number_of_clients + 1, 0.6 * capacity_of_vehicle * number_of_vehicles)

      obj_road, paths_road, edges_road = vrpsdp(number_of_clients, number_of_vehicles, cost_R, demand_of_client_delivery, demand_of_client_pickup, file_idx, "R")

      objectiv_function_road[c_iterator][v_iterator].append(obj_road)
      all_paths_road[c_iterator][v_iterator].append(paths_road)
      all_uesd_edges_road[c_iterator][v_iterator].append(edges_road)

      obj_time, paths_time, edges_time = vrpsdp(number_of_clients, number_of_vehicles, cost_T, demand_of_client_delivery, demand_of_client_pickup, file_idx, "T")

      objectiv_function_time[c_iterator][v_iterator].append(obj_time)
      all_paths_time[c_iterator][v_iterator].append(paths_time)
      all_uesd_edges_time[c_iterator][v_iterator].append(edges_time)

      obj_real, paths_real, edges_real = vrpsdp(number_of_clients, number_of_vehicles, cost_G, demand_of_client_delivery, demand_of_client_pickup, file_idx, "G")

      objectiv_function_real[c_iterator][v_iterator].append(obj_real)
      all_paths_real[c_iterator][v_iterator].append(paths_real)
      all_uesd_edges_real[c_iterator][v_iterator].append(edges_real)

      c_iterator += 1

    v_iterator += 1

print(values_to_reload)

############# calculated_file 30_clients_cracow_1
calculate for 10 clients and 1 vechicles
[[0, 8, 6, 4, 2, 1, 5, 10, 3, 7, 9, 0]]
[[0, 8, 6, 4, 2, 1, 5, 7, 9, 3, 10, 0]]
[[0, 5, 10, 3, 7, 9, 8, 6, 4, 1, 2, 0]]
calculate for 10 clients and 1 vechicles
[[0, 8, 6, 4, 2, 1, 5, 10, 3, 7, 9, 0]]
[[0, 2, 1, 5, 10, 3, 7, 9, 8, 6, 4, 0]]
[[0, 2, 1, 4, 6, 8, 9, 7, 3, 10, 5, 0]]
calculate for 10 clients and 1 vechicles
[[0, 8, 6, 4, 2, 1, 5, 10, 3, 7, 9, 0]]
[[0, 8, 6, 4, 2, 1, 5, 10, 3, 7, 9, 0]]
[]
calculate for 10 clients and 2 vechicles
[[0, 9, 7, 3, 10, 1, 5, 0], [0, 2, 4, 6, 8, 0]]
[[0, 5, 1, 2, 4, 6, 8, 0], [0, 9, 3, 7, 10, 0]]
[[0, 2, 1, 4, 6, 8, 0], [0, 3, 9, 7, 10, 5, 0]]
calculate for 10 clients and 2 vechicles
[[0, 2, 4, 6, 8, 0], [0, 9, 7, 3, 10, 1, 5, 0]]
[[0, 2, 1, 5, 10, 3, 7, 9, 0], [0, 4, 6, 8, 0]]
[[0, 5, 10, 7, 9, 3, 0], [0, 8, 6, 4, 1, 2, 0]]
calculate for 10 clients and 2 vechicles
[[0, 9, 7, 3, 10, 5, 0], [0, 8, 6, 4, 2, 1, 0]]
[[0, 8, 6, 4, 0], [0, 2, 1, 5, 10, 7, 3, 9, 0

KeyboardInterrupt: 

In [ ]:
percent = 0.62

while len(values_to_reload) != 0:
  values_to_reload_mew = []
  percent = percent - 0.02
  print(f"###### Failed roads {len(values_to_reload)}")
  for (number_of_clients, number_of_vehicles, file_idx, problem) in values_to_reload:
    print(f"{number_of_clients}, {number_of_vehicles}, {file_idx}, {problem}")
    df = pd.read_csv(os.path.join(output_dir, files[file_idx]))
    df.fillna(10000000, inplace=True)
    cost_road, cost_time, cost_real = create_cost_matrixes(df)
    demand_of_client_delivery = generate_list(number_of_clients + 1, 0.7 * capacity_of_vehicle * number_of_vehicles)
    demand_of_client_pickup = generate_list(number_of_clients + 1, 0.7 * capacity_of_vehicle * number_of_vehicles)


    obj_1, paths, edges = vrpsdp_error(number_of_clients, number_of_vehicles, cost_road, demand_of_client_delivery, demand_of_client_pickup, file_idx)
    if obj_1 != None:
        objectiv_function_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_1
        all_paths_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_2, paths, edges = vrpsdp_error(number_of_clients, number_of_vehicles, cost_time, demand_of_client_delivery, demand_of_client_pickup, file_idx)
    if obj_2 != None:
        objectiv_function_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_2
        all_paths_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_3, paths, edges = vrpsdp_error(number_of_clients, number_of_vehicles, cost_real, demand_of_client_delivery, demand_of_client_pickup, file_idx)
    if obj_3 != None:
        objectiv_function_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_3
        all_paths_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    if obj_1 == None or obj_2 == None or obj_3 == None:
      values_to_reload_mew.append((number_of_clients, number_of_vehicles, file_idx, problem))

  values_to_reload = values_to_reload_mew


In [ ]:
R_T_edges_preperation = [[[] for _ in range(3)] for _ in range(3)]
R_G_edges_preperation = [[[] for _ in range(3)] for _ in range(3)]
G_T_edges_preperation = [[[] for _ in range(3)] for _ in range(3)]
R_T_paths_preperation = [[[] for _ in range(3)] for _ in range(3)]
R_G_paths_preperation = [[[] for _ in range(3)] for _ in range(3)]
G_T_paths_preperation = [[[] for _ in range(3)] for _ in range(3)]


for file_idx in range(len(files)):
  for vechicles_idx in range(3):
    for clients_idx in range(2):

      R_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))
      R_G_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_real[clients_idx][vechicles_idx][file_idx]))
      G_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_real[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))

      R_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))
      R_G_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_real[clients_idx][vechicles_idx][file_idx]))
      G_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_real[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))



In [ ]:
import numpy as np

AVG_R = [[[] for _ in range(3)] for _ in range(3)]
AVG_R_stdev = [[[] for _ in range(3)] for _ in range(3)]

AVG_G = [[[] for _ in range(3)] for _ in range(3)]
AVG_G_stdev = [[[] for _ in range(3)] for _ in range(3)]

AVG_T = [[[] for _ in range(3)] for _ in range(3)]
AVG_T_stdev = [[[] for _ in range(3)] for _ in range(3)]

R_T_edges = [[[] for _ in range(3)] for _ in range(3)]
R_T_edges_stdev = [[[] for _ in range(3)] for _ in range(3)]

R_G_edges = [[[] for _ in range(3)] for _ in range(3)]
R_G_edges_stdev = [[[] for _ in range(3)] for _ in range(3)]

G_T_edges = [[[] for _ in range(3)] for _ in range(3)]
G_T_edges_stdev = [[[] for _ in range(3)] for _ in range(3)]

R_T_paths = [[[] for _ in range(3)] for _ in range(3)]
R_G_paths = [[[] for _ in range(3)] for _ in range(3)]
G_T_paths = [[[] for _ in range(3)] for _ in range(3)]



for vechicles_idx in range(3):
  for clients_idx in range(2):
     AVG_R[clients_idx][vechicles_idx] = np.mean(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_R_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_T[clients_idx][vechicles_idx] = np.mean(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_T_stdev [clients_idx][vechicles_idx] = np.std(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_G[clients_idx][vechicles_idx] = np.mean(objectiv_function_real[clients_idx][vechicles_idx])
     AVG_G_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_real[clients_idx][vechicles_idx])

     R_T_edges[clients_idx][vechicles_idx] = np.mean(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_T_edges_stdev[clients_idx][vechicles_idx] = np.std(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges[clients_idx][vechicles_idx] = np.mean(R_G_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges_stdev[clients_idx][vechicles_idx] = np.std(R_G_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges[clients_idx][vechicles_idx] = np.mean(G_T_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges_stdev[clients_idx][vechicles_idx] = np.std(G_T_edges_preperation[clients_idx][vechicles_idx])

     R_T_paths[clients_idx][vechicles_idx] = sum(R_T_paths_preperation[clients_idx][vechicles_idx])
     R_G_paths[clients_idx][vechicles_idx] = sum(R_G_paths_preperation[clients_idx][vechicles_idx])
     G_T_paths[clients_idx][vechicles_idx] = sum(G_T_paths_preperation[clients_idx][vechicles_idx])


In [ ]:
print(f"AVG_R = {AVG_R}")
print(f"AVG_R_stdev = {AVG_R_stdev}")
print(f"AVG_T = {AVG_T}")
print(f"AVG_T_stdev = {AVG_T_stdev}")
print(f"AVG_G = {AVG_G}")
print(f"AVG_G_stdev = {AVG_G_stdev}")

print(f"R_T_edges = {R_T_edges}")
print(f"R_T_edges_stdev = {R_T_edges_stdev}")
print(f"R_G_edges = {R_G_edges}")
print(f"R_G_edges_stdev = {R_G_edges_stdev}")
print(f"G_T_edges = {G_T_edges}")
print(f"G_T_edges_stdev = {G_T_edges_stdev}")

print(f"R_T_paths = {R_T_paths}")
print(f"R_G_paths = {R_G_paths}")
print(f"G_T_paths = {G_T_paths}")

AVG_R = [[59.36985000000281, 62.044849874441546, 65.40484999763433], [41.30500000000001, 44.225, 48.325], [[], [], []]]
AVG_R_stdev = [[5.784433803533438, 5.7537510505113385, 6.02231433643269], [10.655678063830571, 10.80184590706607, 10.868158767703008], [[], [], []]]
AVG_T = [[138.69999999973749, 147.84999995666155, 157.64999921021771], [92.79999999999998, 102.80000000002494, 115.05], [[], [], []]]
AVG_T_stdev = [[12.75186260904191, 12.920042555348621, 13.624517425662015], [20.036466754395597, 20.745119908047336, 20.72794008096317], [[], [], []]]
AVG_G = [[24.025000000000194, 25.96749999999988, 27.900999980251573], [18.56299999999999, 20.543999886248848, 23.363999999999997], [[], [], []]]
AVG_G_stdev = [[1.7854453226023197, 1.9931981211108651, 2.2417357281446946], [2.305300631154202, 2.4525588691141387, 3.02247812233604], [[], [], []]]
R_T_edges = [[0.33870413292827084, 0.4170474530628937, 0.3821757433058155], [0.4623803827751196, 0.4121031746031746, 0.6295183946488294], [[], [], []]]

### EUROPE

In [ ]:
# ['30_clients_europe_1', '30_clients_europe_2', '30_clients_europe_3', '30_clients_europe_4', '30_clients_europe_5', '30_clients_europe_7', '30_clients_europe_8', '30_clients_europe_6', '30_clients_europe_9', '30_clients_europe_10', '30_clients_europe_11', '30_clients_europe_12', '30_clients_europe_13', '30_clients_europe_14', '30_clients_europe_15', '30_clients_europe_16', '30_clients_europe_17', '30_clients_europe_18', '30_clients_europe_19', '30_clients_europe_20']

['30_clients_europe_1', '30_clients_europe_2', '30_clients_europe_3', '30_clients_europe_4', '30_clients_europe_5', '30_clients_europe_7', '30_clients_europe_8', '30_clients_europe_6', '30_clients_europe_9', '30_clients_europe_10', '30_clients_europe_11', '30_clients_europe_12', '30_clients_europe_13', '30_clients_europe_14', '30_clients_europe_15', '30_clients_europe_16', '30_clients_europe_17', '30_clients_europe_18', '30_clients_europe_19', '30_clients_europe_20']


In [ ]:
capacity_of_vehicle = 100

objectiv_function_road = [[[] for _ in range(3)] for _ in range(1)]
objectiv_function_time = [[[] for _ in range(3)] for _ in range(1)]
objectiv_function_real = [[[] for _ in range(3)] for _ in range(1)]
all_paths_road = [[[] for _ in range(3)] for _ in range(1)]
all_paths_time = [[[] for _ in range(3)] for _ in range(1)]
all_paths_real = [[[] for _ in range(3)] for _ in range(1)]
all_uesd_edges_road = [[[] for _ in range(3)] for _ in range(1)]
all_uesd_edges_time = [[[] for _ in range(3)] for _ in range(1)]
all_uesd_edges_real = [[[] for _ in range(3)] for _ in range(1)]

values_to_reload = []
vechicles = [1,2,3]
clients = [16]

for requested_file in files:
  print(f"############# calculated_file {requested_file}")
  file_idx = files.index(requested_file)
  df = pd.read_csv(os.path.join(output_dir, requested_file))
  df.fillna(10000000, inplace=True)
  cost_R, cost_T, cost_G = create_cost_matrixes(df)

  v_iterator = 0

  for number_of_vehicles in vechicles:
    c_iterator = 0

    for number_of_clients in clients:

      print(f"calculate for {number_of_clients} clients and {number_of_vehicles} vechicles")

      demand_of_client_delivery = generate_list(number_of_clients + 1, 0.7 * capacity_of_vehicle * number_of_vehicles)
      demand_of_client_pickup = generate_list(number_of_clients + 1, 0.7 * capacity_of_vehicle * number_of_vehicles)

      obj_road, paths_road, edges_road = vrpsdp(number_of_clients, number_of_vehicles, cost_R, demand_of_client_delivery, demand_of_client_pickup, file_idx, "R")

      objectiv_function_road[c_iterator][v_iterator].append(obj_road)
      all_paths_road[c_iterator][v_iterator].append(paths_road)
      all_uesd_edges_road[c_iterator][v_iterator].append(edges_road)

      obj_time, paths_time, edges_time = vrpsdp(number_of_clients, number_of_vehicles, cost_T, demand_of_client_delivery, demand_of_client_pickup, file_idx, "T")

      objectiv_function_time[c_iterator][v_iterator].append(obj_time)
      all_paths_time[c_iterator][v_iterator].append(paths_time)
      all_uesd_edges_time[c_iterator][v_iterator].append(edges_time)

      obj_real, paths_real, edges_real = vrpsdp(number_of_clients, number_of_vehicles, cost_G, demand_of_client_delivery, demand_of_client_pickup, file_idx, "G")

      objectiv_function_real[c_iterator][v_iterator].append(obj_real)
      all_paths_real[c_iterator][v_iterator].append(paths_real)
      all_uesd_edges_real[c_iterator][v_iterator].append(edges_real)

      c_iterator += 1

    v_iterator += 1

print(values_to_reload)

############# calculated_file 30_clients_europe_1
calculate for 16 clients and 1 vechicles
calculate for 16 clients and 2 vechicles
calculate for 16 clients and 3 vechicles
############# calculated_file 30_clients_europe_2
calculate for 16 clients and 1 vechicles
calculate for 16 clients and 2 vechicles
calculate for 16 clients and 3 vechicles
############# calculated_file 30_clients_europe_3
calculate for 16 clients and 1 vechicles
calculate for 16 clients and 2 vechicles
calculate for 16 clients and 3 vechicles
############# calculated_file 30_clients_europe_4
calculate for 16 clients and 1 vechicles
calculate for 16 clients and 2 vechicles
calculate for 16 clients and 3 vechicles
############# calculated_file 30_clients_europe_5
calculate for 16 clients and 1 vechicles
calculate for 16 clients and 2 vechicles
calculate for 16 clients and 3 vechicles
############# calculated_file 30_clients_europe_7
calculate for 16 clients and 1 vechicles
calculate for 16 clients and 2 vechicles
cal

In [ ]:
percent = 0.62

while len(values_to_reload) != 0:
  values_to_reload_mew = []
  percent = percent - 0.02
  print(f"###### Failed roads {len(values_to_reload)}")
  for (number_of_clients, number_of_vehicles, file_idx, problem) in values_to_reload:
    print(f"{number_of_clients}, {number_of_vehicles}, {file_idx}, {problem}")
    df = pd.read_csv(os.path.join(output_dir, files[file_idx]))
    df.fillna(10000000, inplace=True)
    cost_road, cost_time, cost_real = create_cost_matrixes(df)
    demand_of_client_delivery = generate_list(number_of_clients + 1, 0.7 * capacity_of_vehicle * number_of_vehicles)
    demand_of_client_pickup = generate_list(number_of_clients + 1, 0.7 * capacity_of_vehicle * number_of_vehicles)


    obj_1, paths, edges = vrpsdp_error(number_of_clients, number_of_vehicles, cost_road, demand_of_client_delivery, demand_of_client_pickup, file_idx)
    if obj_1 != None:
        objectiv_function_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_1
        all_paths_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_2, paths, edges = vrpsdp_error(number_of_clients, number_of_vehicles, cost_time, demand_of_client_delivery, demand_of_client_pickup, file_idx)
    if obj_2 != None:
        objectiv_function_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_2
        all_paths_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_3, paths, edges = vrpsdp_error(number_of_clients, number_of_vehicles, cost_real, demand_of_client_delivery, demand_of_client_pickup, file_idx)
    if obj_3 != None:
        objectiv_function_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_3
        all_paths_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    if obj_1 == None or obj_2 == None or obj_3 == None:
      values_to_reload_mew.append((number_of_clients, number_of_vehicles, file_idx, problem))

  values_to_reload = values_to_reload_mew


In [ ]:
R_T_edges_preperation = [[[] for _ in range(3)] for _ in range(1)]
R_G_edges_preperation = [[[] for _ in range(3)] for _ in range(1)]
G_T_edges_preperation = [[[] for _ in range(3)] for _ in range(1)]
R_T_paths_preperation = [[[] for _ in range(3)] for _ in range(1)]
R_G_paths_preperation = [[[] for _ in range(3)] for _ in range(1)]
G_T_paths_preperation = [[[] for _ in range(3)] for _ in range(1)]


for file_idx in range(len(files)):
  for vechicles_idx in range(3):
    for clients_idx in range(1):

      R_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))
      R_G_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_real[clients_idx][vechicles_idx][file_idx]))
      G_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_real[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))

      R_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))
      R_G_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_real[clients_idx][vechicles_idx][file_idx]))
      G_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_real[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))



In [ ]:
import numpy as np

AVG_R = [[[] for _ in range(3)] for _ in range(1)]
AVG_R_stdev = [[[] for _ in range(3)] for _ in range(1)]

AVG_G = [[[] for _ in range(3)] for _ in range(1)]
AVG_G_stdev = [[[] for _ in range(3)] for _ in range(1)]

AVG_T = [[[] for _ in range(3)] for _ in range(1)]
AVG_T_stdev = [[[] for _ in range(3)] for _ in range(1)]

R_T_edges = [[[] for _ in range(3)] for _ in range(1)]
R_T_edges_stdev = [[[] for _ in range(3)] for _ in range(1)]

R_G_edges = [[[] for _ in range(3)] for _ in range(1)]
R_G_edges_stdev = [[[] for _ in range(3)] for _ in range(1)]

G_T_edges = [[[] for _ in range(3)] for _ in range(1)]
G_T_edges_stdev = [[[] for _ in range(3)] for _ in range(1)]

R_T_paths = [[[] for _ in range(3)] for _ in range(1)]
R_G_paths = [[[] for _ in range(3)] for _ in range(1)]
G_T_paths = [[[] for _ in range(3)] for _ in range(1)]


for vechicles_idx in range(3):
  for clients_idx in range(1):
     AVG_R[clients_idx][vechicles_idx] = np.mean(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_R_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_T[clients_idx][vechicles_idx] = np.mean(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_T_stdev [clients_idx][vechicles_idx] = np.std(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_G[clients_idx][vechicles_idx] = np.mean(objectiv_function_real[clients_idx][vechicles_idx])
     AVG_G_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_real[clients_idx][vechicles_idx])

     R_T_edges[clients_idx][vechicles_idx] = np.mean(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_T_edges_stdev[clients_idx][vechicles_idx] = np.std(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges[clients_idx][vechicles_idx] = np.mean(R_G_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges_stdev[clients_idx][vechicles_idx] = np.std(R_G_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges[clients_idx][vechicles_idx] = np.mean(G_T_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges_stdev[clients_idx][vechicles_idx] = np.std(G_T_edges_preperation[clients_idx][vechicles_idx])

     R_T_paths[clients_idx][vechicles_idx] = sum(R_T_paths_preperation[clients_idx][vechicles_idx])
     R_G_paths[clients_idx][vechicles_idx] = sum(R_G_paths_preperation[clients_idx][vechicles_idx])
     G_T_paths[clients_idx][vechicles_idx] = sum(G_T_paths_preperation[clients_idx][vechicles_idx])


In [ ]:
print(f"AVG_R = {AVG_R}")
print(f"AVG_R_stdev = {AVG_R_stdev}")
print(f"AVG_T = {AVG_T}")
print(f"AVG_T_stdev = {AVG_T_stdev}")
print(f"AVG_G = {AVG_G}")
print(f"AVG_G_stdev = {AVG_G_stdev}")

print(f"R_T_edges = {R_T_edges}")
print(f"R_T_edges_stdev = {R_T_edges_stdev}")
print(f"R_G_edges = {R_G_edges}")
print(f"R_G_edges_stdev = {R_G_edges_stdev}")
print(f"G_T_edges = {G_T_edges}")
print(f"G_T_edges_stdev = {G_T_edges_stdev}")

print(f"R_T_paths = {R_T_paths}")
print(f"R_G_paths = {R_G_paths}")
print(f"G_T_paths = {G_T_paths}")


AVG_R = [[4066.234999999929, 4645.044999999999, 5288.029999999997]]
AVG_R_stdev = [[476.2125610219358, 569.0014274806362, 614.984181991718]]
AVG_T = [[2953.749999999995, 3322.950000000099, 3729.4499999999803]]
AVG_T_stdev = [[326.2397699545501, 365.17139469018196, 380.77230400857326]]
AVG_G = [[2991.726500000007, 3431.7419999999997, 3936.563497887398]]
AVG_G_stdev = [[338.4855784265525, 390.50398953915885, 426.6730284076523]]
R_T_edges = [[0.3677133823748935, 0.5138749754356965, 0.5283920923133352]]
R_T_edges_stdev = [[0.3986625236496098, 0.3019997958069016, 0.2557296394395905]]
R_G_edges = [[0.4493446596029715, 0.47531473591080886, 0.36871289474168195]]
R_G_edges_stdev = [[0.39050014948197564, 0.288176194365624, 0.19911887612586646]]
G_T_edges = [[0.46120043161869867, 0.4002472731516849, 0.4632412182582327]]
G_T_edges_stdev = [[0.3635848294387955, 0.25349914460397155, 0.31049156989519616]]
R_T_paths = [[7, 1, 1]]
R_G_paths = [[5, 1, 1]]
G_T_paths = [[6, 1, 1]]


### RESULTS

In [ ]:
for client in clients:
  for vechicle in vechicles:
    df.loc[iterator] = [topology, client, vechicle,
                        str(AVG_R[clients.index(client)][vechicles.index(vechicle)]) + " ± " + str(plus_minus(AVG_R_stdev[clients.index(client)][vechicles.index(vechicle)])),
                        str(AVG_G[clients.index(client)][vechicles.index(vechicle)]) + " ± " + str(plus_minus(AVG_G_stdev[clients.index(client)][vechicles.index(vechicle)])),
                        str(AVG_T[clients.index(client)][vechicles.index(vechicle)]) + " ± " + str(plus_minus(AVG_T_stdev[clients.index(client)][vechicles.index(vechicle)])),
                        str(R_T_edges[clients.index(client)][vechicles.index(vechicle)]) + "% ± " + str(plus_minus(R_T_edges_stdev[clients.index(client)][vechicles.index(vechicle)])) +"%",
                        str(R_G_edges[clients.index(client)][vechicles.index(vechicle)]) + "% ± " + str(plus_minus(R_G_edges_stdev[clients.index(client)][vechicles.index(vechicle)])) +"%",
                        str(G_T_edges[clients.index(client)][vechicles.index(vechicle)]) + "% ± " + str(plus_minus(G_T_edges_stdev[clients.index(client)][vechicles.index(vechicle)])) +"%",
                        R_T_paths[clients.index(client)][vechicles.index(vechicle)], R_G_paths[clients.index(client)][vechicles.index(vechicle)], G_T_paths[clients.index(client)][vechicles.index(vechicle)]
                        ]
    iterator += 1


In [ ]:
| Topology | No. clients | No. vehicles | Road distance matrix | Geographical distance matrix | Time distance matrix |
|----------|-------------|--------------|----------------------|-----------------------------|----------------------|
| Krakow   | 10          | 1            | 41.305 ± 4.987       | 18.563 ± 1.079              | 92.8 ± 9.377         |
| Krakow   | 10          | 2            | 44.22 ± 4.997        | 20.574 ± 1.062              | 103.0 ± 9.576        |
| Krakow   | 10          | 3            | 48.165 ± 5.317       | 23.182 ± 1.322              | 114.25 ± 10.026      |
| Krakow   | 15          | 1            | 53.764 ± 3.758       | 21.519 ± 0.915              | 122.05 ± 7.917       |
| Krakow   | 15          | 2            | 56.409 ± 3.863       | 23.521 ± 0.966              | 131.5 ± 8.194        |
| Krakow   | 15          | 3            | 60.239 ± 3.76        | 26.065 ± 1.13               | 142.75 ± 7.865       |
| Krakow   | 20          | 1            | 59.37 ± 2.707        | 24.025 ± 0.835              | 138.7 ± 5.968        |
| Krakow   | 20          | 2            | 62.045 ± 2.693       | 25.967 ± 0.933              | 147.85 ± 6.047       |
| Krakow   | 20          | 3            | 65.405 ± 2.818       | 27.901 ± 1.049              | 157.65 ± 6.377       |
| Europe   | 10          | 1            | 3261.11 ± 203.988    | 2426.368 ± 143.128          | 2279.55 ± 142.798    |
| Europe   | 10          | 2            | 3726.425 ± 236.205   | 2769.135 ± 159.045          | 2567.0 ± 162.517     |
| Europe   | 10          | 3            | 4296.72 ± 292.268    | 3199.232 ± 208.09           | 2935.6 ± 191.515     |
| Europe   | 15          | 1            | 3918.415 ± 217.34    | 2887.159 ± 164.53           | 2832.05 ± 151.968    |
| Europe   | 15          | 2            | 4383.12 ± 243.633    | 3230.775 ± 171.873          | 3113.55 ± 156.163    |
| Europe   | 15          | 3            | 4817.955 ± 267.938   | 3583.803 ± 183.617          | 3396.55 ± 166.79     |
| Europe   | 20          | 1            | 4503.135 ± 155.306   | 3277.371 ± 103.334          | 3323.4 ± 107.992     |
| Europe   | 20          | 2            | 4890.39 ± 182.296    | 3577.708 ± 115.713          | 3577.5 ± 124.705     |
| Europe   | 20          | 3            | 5322.895 ± 214.345   | 3913.567 ± 147.687          | 3855.25 ± 128.705    |


In [ ]:
| Topology | No. clients | No. vehicles | R/T e. [\%]       | R/G e. [\%]       | G/T e. [\%]       | R/T p. | R/G p. | G/T p. |
|----------|-------------|--------------|-------------------|-------------------|-------------------|--------|--------|--------|
| Krakow   | 10          | 1            | 38.18 ± 18.38      | 30.46 ± 12.99      | 32.73 ± 12.86      | 4      | 1      | 1      |
| Krakow   | 10          | 2            | 44.75 ± 14.53      | 33.75 ± 14.04      | 42.17 ± 12.09      | 3      | 1      | 2      |
| Krakow   | 10          | 3            | 47.31 ± 12.28      | 45.77 ± 8.99       | 39.62 ± 11.11      | 2      | 1      | 1      |
| Krakow   | 15          | 1            | 45.00 ± 16.28      | 38.44 ± 11.15      | 30.94 ± 11.12      | 2      | 0      | 0      |
| Krakow   | 15          | 2            | 45.88 ± 13.48      | 33.53 ± 9.85       | 30.59 ± 8.24       | 2      | 0      | 0      |
| Krakow   | 15          | 3            | 55.00 ± 8.70       | 32.78 ± 10.49      | 34.72 ± 5.69       | 1      | 0      | 0      |
| Krakow   | 20          | 1            | 33.87 ± 12.61      | 15.59 ± 7.03       | 18.53 ± 9.77       | 1      | 0      | 0      |
| Krakow   | 20          | 2            | 41.71 ± 11.07      | 32.12 ± 9.96       | 26.25 ± 6.76       | 0      | 0      | 0      |
| Krakow   | 20          | 3            | 38.22 ± 8.91       | 28.48 ± 7.33       | 30.00 ± 7.48       | 1      | 0      | 0      |
| Europe   | 10          | 1            | 86.20 ± 10.24      | 74.18 ± 20.40      | 68.10 ± 18.90      | 16     | 13     | 10     |
| Europe   | 10          | 2            | 85.33 ± 12.72      | 70.17 ± 12.88      | 65.92 ± 15.77      | 14     | 11     | 9      |
| Europe   | 10          | 3            | 66.20 ± 12.40      | 58.73 ± 11.89      | 57.97 ± 11.27      | 4      | 4      | 3      |
| Europe   | 15          | 1            | 54.64 ± 19.85      | 54.58 ± 19.29      | 57.47 ± 18.51      | 6      | 6      | 7      |
| Europe   | 15          | 2            | 56.19 ± 15.89      | 52.30 ± 14.85      | 44.05 ± 16.67      | 3      | 3      | 3      |
| Europe   | 15          | 3            | 56.06 ± 10.33      | 57.35 ± 13.47      | 51.23 ± 12.29      | 3      | 2      | 2      |
| Europe   | 20          | 1            | 14.73 ± 28.14      | 11.80 ± 12.58      | 20.76 ± 12.58      | 4      | 2      | 2      |
| Europe   | 20          | 2            | 50.77 ± 14.62      | 44.12 ± 11.56      | 45.09 ± 12.05      | 2      | 2      | 2      |
| Europe   | 20          | 3            | 47.66 ± 13.78      | 43.35 ± 13.19      | 41.46 ± 12.50      | 3      | 3      | 1      |
